## Calculate ATO for TAZ centroid(s)

1. Run 1_setup_network.ipynb to create NetworkDataset_MM
2. Create OD Cost Matrix Layer
3. Add origin(s)
4. Add destinations
5. Solve
6. Join attributes to solved Lines layer
7. Weight HH and JOB by time decay
8. Sum

In [ ]:
import arcpy
import os

arcpy.CheckOutExtension("network")

from arcgis.features import SpatialDataFrame
import pandas as pd
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

base_path = os.path.abspath(".")
base_gdb = os.path.join(base_path, "base.gdb")

arcpy.env.workspace = base_gdb

# Set to True to limit to ~30 TAZs
testing = True

# Set to "scenario_auto" to evaluate the mod_drive network
network = "baseline"
#network = "mod"

#mode = "Driving" 
mode = "Transit"

In [ ]:
if testing:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_sample")
else:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_snapped")

output_file = mode + '_' + network
if network == "baseline":
    # baseline
    input_network_dataset = os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND")
elif network == "mod":
    # mod
    target_gdb =  os.path.join(base_path, "mod.gdb")
    arcpy.env.workspace = target_gdb
    input_network_dataset = os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND")
else:
    print("Please choose a valid input")

In [ ]:
# Create OD Cost Matrix Layers
arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source = input_network_dataset, 
    layer_name = "OD Cost Matrix", 
    travel_mode = mode, 
    cutoff = 60.0,
    line_shape = "NO_LINES"
)

In [ ]:
# add ODs for simple network
arcpy.na.AddLocations(
    "OD Cost Matrix", 
    "Origins",
    centroids, 
    "Name CO_TAZID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Length # #;Cutoff_Mins_DriveTime # #", 
    "10000 Meters"
)
arcpy.na.AddLocations(
    "OD Cost Matrix", 
    "Destinations", 
    centroids, 
    "Name CO_TAZID #;CurbApproach # 0", 
    "10000 Meters"
)

In [ ]:
%%time
# Solve
arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')

In [ ]:
%%time
od = pd.DataFrame.spatial.from_featureclass(r"OD Cost Matrix\Lines")

In [ ]:
od.head()

In [ ]:
# if the network is broken this will fail
# if fails, try deleting and rebuilding network - this usually fixes things. Esri weirdness!
assert od['Total_Mins_DriveOnly'].sum() > 100 or od['Total_Mins_PedTransitOnly'].sum() > 100

In [ ]:
%%time
od['Origin_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[0]))
od['Dest_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[1]))

In [ ]:
taz = pd.DataFrame.spatial.from_featureclass(os.path.join(r"shp\taz_wfrc.gdb", "ATO"))

In [ ]:
taz = taz[['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

In [ ]:
df = pd.merge(od, taz, left_on="Dest_CO_TAZID", right_on="CO_TAZID", )

In [ ]:
# Weight outputs

if mode == 'Driving':
    df['travel_time'] = df['Total_Mins_DriveOnly']
elif mode == 'Transit':
    df['travel_time'] = df['Total_Mins_PedTransitOnly']

df['survey_weight'] = df['travel_time'].apply(lambda x: survey_weight(x))
df['survey_weight'] = df['survey_weight'].round(3)

df['weighted_jobs'] = df['survey_weight'] * df['JOB_19']
df['weighted_hh'] = df['survey_weight'] * df['HH_19']
df['weighted_jobs'] = round(df['weighted_jobs'])
df['weighted_hh'] = round(df['weighted_hh'])

df['ato'] = df['weighted_jobs'] + df['weighted_hh']

In [ ]:
# write to disk
df[['Name', 'Origin_CO_TAZID', 'Dest_CO_TAZID', 'travel_time', 'survey_weight',
         'weighted_jobs', 'weighted_hh', 'ato']].to_csv(output_file + '.csv')

In [ ]:
taz_summary = df.groupby('Origin_CO_TAZID').agg(
    jobs=pd.NamedAgg(column='weighted_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='weighted_hh', aggfunc=sum)
)
taz_summary['ato'] = taz_summary['jobs'] + taz_summary['hh']
taz_summary.to_csv(output_file + '_summary.csv')